In [ ]:
import os
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [ ]:
# for class_name in classes:
#     class_path = os.path.join('assets', class_name)
#     images = os.listdir(class_path)
#     train_images, test_images = train_test_split(images, test_size= 0.2, random_state= 42)
#     train_class_dir = os.path.join(train_dir, class_name)
#     test_class_dir = os.path.join(test_dir, class_name)
#     os.makedirs(train_class_dir, exist_ok= True)
#     os.makedirs(test_class_dir, exist_ok= True)

#     for image in train_images:
#         src = os.path.join(class_path, image)
#         dst = os.path.join(train_class_dir, image)
#         shutil.copy(src, dst)

#     for image in test_images:
#         src = os.path.join(class_path, image)
#         dst = os.path.join(test_class_dir, image)
#         shutil.copy(src, dst)


In [ ]:
# train_datagen = ImageDataGenerator()
# X_train = train_datagen.flow_from_directory(train_dir, target_size= (28, 28), class_mode="sparse")


In [ ]:
# test_datagen = ImageDataGenerator()
# X_test = test_datagen.flow_from_directory(test_dir, target_size= (28, 28), class_mode= "sparse")


In [ ]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    "assets",
    labels = "inferred",
    label_mode = "int",
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (28, 28),
    validation_split = 0.2,
    subset = "both",
    seed = 2,
    )

## Creating a CNN

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))
cnn.summary()

In [ ]:
cnn.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
cnn.fit(x= train, validation_data= validation, epochs=10)


In [ ]:
cnn.save("number_identifier")

In [ ]:
from keras.preprocessing import image
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from keras.models import load_model
import cv2
import numpy as np
nums = []
cnn = load_model('number_identifier')
for i in range(81):
    path = "number_images/" + str(i) + ".jpg"
    test_image = image.load_img(path, target_size = (28, 28))
    test_image = image.img_to_array(test_image)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    test_image = np.expand_dims(test_image, axis = 0)
    result = list(cnn.predict(test_image, verbose= 0)[0])
    nums.append(result.index(max(result)))
nums = np.reshape(nums, (9,9))
print(nums)